In [ ]:
import pandas as pd
import numpy as np
import time
import csv
from sklearn.preprocessing import StandardScaler
import joblib

# IMO Constants
FUEL_DENSITY = 0.991       # kg/L
CO2_FACTOR = 3.114         # g CO2/g fuel
GT = 14052                 # Gross Tonnage of RO-RO ship

# Load and sort dataset
data = pd.read_csv("sample_data_year.csv", low_memory=False)
data['Time'] = pd.to_datetime(data['Time'], format='%d-%m-%Y %H:%M')
data = data.sort_values('Time').reset_index(drop=True)

# Clean numeric columns
cols = ['FO_ME_Cons', 'Ship_Speed', 'CppPitch', 'Wind_Speed', 'HEEL', 'Fore_Draft', 'Aft_Draft']
for col in cols:
    data[col] = pd.to_numeric(data[col], errors='coerce').ffill()

# Drop missing and idle rows
data = data.dropna()
data = data[data['Ship_Speed'] > 0.5]

# Feature engineering
data['Trim'] = data['Aft_Draft'] - data['Fore_Draft']
data['Avg_Draft'] = (data['Fore_Draft'] + data['Aft_Draft']) / 2
data['Fuel_Liters'] = data['FO_ME_Cons'].diff().clip(lower=0)

# Load trained model and scaler
rf_model = joblib.load('rf_final_model.joblib')
scaler = joblib.load('scaler.joblib')

# Feature columns
features = ['CO2_Emission_g', 'Distance_NM', 'Avg_Speed', 'Avg_Wind_Speed',
            'Avg_CppPitch', 'Avg_Heel', 'Avg_Trim', 'Avg_Draft']

# Suggestion generator
def generate_suggestion(row):
    suggestions = []
    if row['Trim'] > 1.0:
        suggestions.append("Reduce trim to improve fuel efficiency")
    if row['HEEL'] > 0.5:
        suggestions.append("Balance ballast to reduce heel")
    if row['Wind_Speed'] > 12:
        suggestions.append("Avoid sailing during high wind")
    if row['CppPitch'] < 15:
        suggestions.append("Increase CPP pitch for propulsion efficiency")
    if row['Ship_Speed'] < 17:
        suggestions.append("Maintain optimal cruising speed")
    return "; ".join(suggestions) if suggestions else "Performance is within expected range"

# CII calculation using GT
def calculate_cii_1min(fuel_liters, ship_speed):
    if ship_speed > 0:
        co2 = fuel_liters * FUEL_DENSITY * 1000 * CO2_FACTOR  # grams
        distance_nm = ship_speed / 60  # NM per minute
        return co2 / (GT * distance_nm) if distance_nm > 0 else np.nan
    else:
        return np.nan

# Initialize CSV log file
log_file = open("cii_simulation_log.csv", mode='w', newline='')
log_writer = csv.DictWriter(log_file, fieldnames=[
    'Minute', 'Time', 'Instant_CII', 'Predicted_CII', 'Suggestions'
])
log_writer.writeheader()

# --- START SIMULATION ---
print("\n --- LIVE CII SIMULATION STARTED ---\n")
cumulative_rows = []

try:
    for i in range(1, len(data)):
        row = data.iloc[i]
        fuel = row['Fuel_Liters']
        speed = row['Ship_Speed']

        # Calculate 1-minute CO2 and distance
        co2_emission = fuel * FUEL_DENSITY * 1000 * CO2_FACTOR
        distance_nm = speed / 60

        # Append to cumulative data
        cumulative_rows.append({
            'CO2_Emission_g': co2_emission,
            'Distance_NM': distance_nm,
            'Avg_Speed': row['Ship_Speed'],
            'Avg_Wind_Speed': row['Wind_Speed'],
            'Avg_CppPitch': row['CppPitch'],
            'Avg_Heel': row['HEEL'],
            'Avg_Trim': row['Trim'],
            'Avg_Draft': row['Avg_Draft']
        })

        cumulative_df = pd.DataFrame(cumulative_rows)

        # Prepare input for model
        processed_input = pd.DataFrame([{
            'CO2_Emission_g': cumulative_df['CO2_Emission_g'].sum(),
            'Distance_NM': cumulative_df['Distance_NM'].sum(),
            'Avg_Speed': cumulative_df['Avg_Speed'].mean(),
            'Avg_Wind_Speed': cumulative_df['Avg_Wind_Speed'].mean(),
            'Avg_CppPitch': cumulative_df['Avg_CppPitch'].mean(),
            'Avg_Heel': cumulative_df['Avg_Heel'].mean(),
            'Avg_Trim': cumulative_df['Avg_Trim'].mean(),
            'Avg_Draft': cumulative_df['Avg_Draft'].mean()
        }])

        scaled_input = scaler.transform(processed_input)
        predicted_cii = rf_model.predict(scaled_input)[0]
        instant_cii = calculate_cii_1min(fuel, speed)
        suggestion = generate_suggestion(row)

        # Terminal output
        print(f" Minute {i}:")
        print(f"    Instant CII     = {instant_cii:.4f}" if not np.isnan(instant_cii) else "    Instant CII     = N/A (zero speed)")
        print(f"    Predicted CII   = {predicted_cii:.4f}")
        print(f"    Suggestions     = {suggestion}")
        print("-" * 50)

        # Real-time logging
        log_writer.writerow({
            'Minute': i,
            'Time': row['Time'],
            'Instant_CII': round(instant_cii, 4) if not np.isnan(instant_cii) else None,
            'Predicted_CII': round(predicted_cii, 4),
            'Suggestions': suggestion
        })

        time.sleep(1)

except KeyboardInterrupt:
    print("\n\n--- SIMULATION STOPPED MANUALLY ---")

finally:
    log_file.close()
    print("\n--- LIVE CII SIMULATION COMPLETE ---")
    print("Output saved to: cii_simulation_log.csv")



 --- LIVE CII SIMULATION STARTED ---

 Minute 1:
    Instant CII     = 19.1521
    Predicted CII   = 48.6993
    Suggestions     = Performance is within expected range
--------------------------------------------------
 Minute 2:
    Instant CII     = 18.3860
    Predicted CII   = 48.9333
    Suggestions     = Performance is within expected range
--------------------------------------------------
 Minute 3:
    Instant CII     = 17.6200
    Predicted CII   = 48.7022
    Suggestions     = Balance ballast to reduce heel
--------------------------------------------------
 Minute 4:
    Instant CII     = 17.6200
    Predicted CII   = 48.7022
    Suggestions     = Performance is within expected range
--------------------------------------------------
 Minute 5:
    Instant CII     = 16.0878
    Predicted CII   = 48.7004
    Suggestions     = Performance is within expected range
--------------------------------------------------
 Minute 6:
    Instant CII     = 17.6200
    Predicted CII   =